# Machine Learning with Signals Problem Set

Per usual, we are going to start by downoloading any packages or data that we need and importing all of the Python packages we will need, too.

In [ ]:
# Install Graphvis library if it is not already installed
!pip install graphviz 

# Clone the audio data repository (should cleanly fail if already downloaded)
!git clone https://github.com/Jakobovski/free-spoken-digit-dataset.git

# Define the path to where the data are downloaded
data_dir = '/work/Week2/04_SignalsML/free-spoken-digit-dataset/recordings/'

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
fatal: destination path 'free-spoken-digit-dataset' already exists and is not an empty directory.


In [ ]:
# Import basic system and numerical packages
import os
import re
import numpy as np
import pandas as pd

# Import statistical and signal processing functions
from scipy import signal
import scipy.stats.mstats as mstats
from sklearn import metrics

# Import the scikit-learn functions and models
from sklearn.model_selection import train_test_split
from sklearn import tree

# Import audio i/o and playing functions
import scipy.io.wavfile
from IPython.display import Audio

# Import plotting functionality
import matplotlib
import matplotlib.pyplot as plt
import graphviz
import seaborn as sns

# Setup the plotting preferencs
%matplotlib inline
matplotlib.rcParams.update({'font.size': 16})

##Problem 1:
$\text{Ben Bitdiddle}$, an amateur data scientist, is working on a difficult classication problem given by his boss $\text{Alyssa P. Hacker}$. Eventually, he comes up with a few features that be beleives are best of the best for his data and plots them below. Based on the visualization in the plot below, which feature is the most useful and which one is the least useful, why? Is Ben really the best?

<img src="https://github.com/BeaverWorksMedlytics2020/Data_Public/raw/master/Images/Week2/good_or_bad_features.png">


X1 is the most useful because there is the least overlap between classes in the distribution plot. X3 is the least useful because it has the most overlap.

##Problem 2

  Here we are revisiting our dataset of recordings of spoken digtis (0-9). Here we will add some twists! Remember to visualize and really think about what features may or may not be useful!

In [ ]:
# Let's double check that get got all of the files we expected
file_list = os.listdir(data_dir)
print('{} audio samples\n'.format(len(file_list)))

# list the first few files
print('Example files...')
print(file_list[0:10])

3000 audio samples

Example files...
['8_nicolas_35.wav', '5_lucas_3.wav', '1_nicolas_23.wav', '2_jackson_32.wav', '6_george_4.wav', '2_nicolas_28.wav', '0_jackson_41.wav', '1_jackson_39.wav', '5_nicolas_14.wav', '7_george_38.wav']


## Feature Engineering
As we saw earlier, some features are more useful for classification than others. Here, we provide you with functions to extract the "spectral centroid" and "spectral flatness" features from an audio file, but we will also help you start to build your own feature extracting functions.

### Spectral Centroid

The [**spectral centroid**](https://en.wikipedia.org/wiki/Spectral_centroid) indicates at which frequency the energy of a spectrum is centered upon. This is like a weighted mean:
$$f_c=\frac{\sum_kA(k)f(k)}{\sum_kA(k)}$$
 
where $A(k)$ is the spectral magnitude at frequency bin $k$,  $f(k)$ is the frequency at bin $k$
 .

In [ ]:
def spectral_centroid(ft, f_s):
    """
    Computes the spectral centroid from the FT of a signal

    :param ft: output of Fourier Transform (i.e., np.fft.fft())
    :param f_s: sampling frequency (or `sampling rate`) (in Hz)
    """
    # Generate the frequencies associated with the fourier transform values
    num_samples = len(ft)
    freqs = np.linspace(0, f_s, num_samples)

    # Grab the magnitude of the relevant part of the fourier transform values
    freqs = freqs[0:num_samples//2]
    magnitude = np.abs(ft[0:num_samples//2]) * (2/num_samples)
    
    # Compute the weighted frequency to get the specral centroid
    spec_centroid = np.sum(magnitude*freqs)/np.sum(magnitude)

    return spec_centroid

### Spectral Flatness

The [**spectral flatness**](https://en.wikipedia.org/wiki/Spectral_flatness), also known as Wiener entropy, is a measure used in digital signal processing to characterize an audio spectrum. Spectral flatness is typically measured in decibels, and provides a way to quantify how noise-like a sound is, as opposed to being tone-like.

In [ ]:
def spectral_flatness(ft):
    """
    Computes the spectral flatness of the FT of a signal
    
    :param ft: output of Fourier Transform (i.e., np.fft.fft())
    """
    # Grab the magnitude of the relevant part of the fourier transform values
    num_samples = len(ft)
    magnitude = abs(ft[0:num_samples//2]) * (2/num_samples)
 
    # Spectral flatness is simply the geometric mean of the FT magnitude
    # divided by the arithmetic mean of the FT magnitude
    spec_flatness = mstats.gmean(magnitude)/np.mean(magnitude)

    return spec_flatness

### Spectral Roll-off
In this section, you are going to write the code to extract a new feature called "spectral roll-off". This measure can used to measure the distinguish voice from non-voice audio. This can be used to give a good idea of what frequencies contain the majority of the energy. Spectral roll-off can be computed from the inequality:

$$\sum^{r}_{k=0} A(k) \geq 0.85\sum^{N}_{k=0} A(k) \text{,}$$

where $A(k)$ is the magnitude of the fourier transform at index $k$.

The idea is to find the frequency associated with index $r$, where the sum to the magnitudes in the fourier transform from $k=0$ up to $k=r$ is greater than or equal to $85$ percent of the sum the magnitudes from $k=0$ to $k=N$.

In [ ]:
def spectral_rolloff(ft, f_s):
    # Generate the frequencies associated with the fourier transform values
    num_samples = len(ft) # <-- UPDATE THESE LINES
    freqs = np.linspace(0, f_s, num_samples)

    # Grab the magnitude of the relevant part of the fourier transform values
    freqs = freqs[:num_samples//2]       # <-- UPDATE THESE LINES
    magnitude = np.abs(ft[:num_samples//2])

    # Compute the sum of *all* the fourier transform magnitudes
    whole_sum = np.sum(magnitude)   # <-- UPDATE THIS LINE

    # Initialize a variable to hold the cumulative sum of the magnitudes of the
    # fourier transform as we loop over it.
    cumulative_sum = 0.0

    # Loop over each element in the fourier transform
    for k, ft_mag in enumerate(magnitude):
        # Add the current magnitude of the cumulative sum
        cumulative_sum += ft_mag     # <-- UPDATE THIS LINE

        # Check if the cumulative sum is currently greater than 85% of the
        # whole sum, and return the frequency at which this inequality first
        # becomes true.
        if cumulative_sum >= 0.85*whole_sum:
            return freqs[k]

#Implement your own feature extractors
Add your own feature! We've added some common techniques based on some popular audio signal processing methods. However, there are plenty out there. The links below contain some information regarding a few possible features to use (including some we have already implemented).

Sourc 1: https://musicinformationretrieval.com/spectral_features.html

Source 2: https://www.cs.ccu.edu.tw/~wtchu/courses/2014f_MCA/Lectures/Lecture%209%20Audio%20and%20Music%20Analysis%202.pdf

There are lots of features to choose from, so pick a few that you think mightc help classify an audio clip (or feel free to create your own ;-) ). 

A few possible features from these sources:
* Spectral Bandwidth
* Spectral Flux
* Zero Crossing Rate
* Spectral Contrast
  - This one is tricky because it requires using some form of the "spectrogram" (not just the fourier transform), but it also contains *much* more information and may be helpful for signal classification.

Other ideas:
* The "Maximum Frequency Value" shown in the tutorial stil exists. We are not using it here, but don't forget it exists.
* Weighted standard deviation of the fourier transform
   - This would be analogous to the spectral centroid but for standard deviation
* Break up frequency into several "frequency subbands", compute centroid within each subband, and use each of those centroid values as a feature
* Use the *number* of different peaks in the fourier transform as a feature
* The sky is the limit, so get creative!


In [ ]:
def spectral_bandwidth_manual(s, f_s, p=2):
    N = len(s)
    f = np.linspace(0, f_s, N)
    fc = spectral_centroid(s, f_s)

    total = 0.0
    for k in range(N):
        total += s[k] * ((f[k] - fc) ** p)
    
    return total ** (1/p)

In [ ]:
import librosa
def feature1_extractor(s, f_s):
    S = []
    for i in s:
        S.append(float(i))
    return np.mean(librosa.feature.spectral_bandwidth(y=np.array(S), sr=float(f_s), n_fft = len(S))[0])

In [ ]:
'''def spec_band(s, f_s):
    S = []
    for i in s:
        S.append(float(i))
    #return librosa.feature.spectral_bandwidth(y=np.array(S), sr=float(f_s), n_fft=)'''

'def spec_band(s, f_s):\n    S = []\n    for i in s:\n        S.append(float(i))\n    #return librosa.feature.spectral_bandwidth(y=np.array(S), sr=float(f_s), n_fft=)'

In [ ]:
def feature2_extractor(s, f_s):
    """The even cooler feature extractor I wrote"""
    Z = 0
    for i in range(1, len(s)):
        Z += abs(np.sign(s[i])-np.sign(s[i-1]))
    return 0.5*Z

## Load full dataset and extract features

In [ ]:
# We can use the regular expression functionality (`re` package) to robustly
# parse the name of each audio file to retrieve the digit spoken, the person
# speaking, and the trial number of that digit-speaker pair.
#
# The following lines create little "string parsers" to grab that information.
re_digit = re.compile('\d+_')
re_speaker = re.compile('_[a-z]+_')
re_trial = re.compile('_\d.')

# Load this information into a dataframe and create columns to hold the features
# Spectral Centroid (SC)
# Spectral Flatness (SF)
##########################################################
# ADD THE NAMES OF YOUR FEATURES TO THIS LIST OF COLUMNS #
##########################################################
df = pd.DataFrame(columns=['digit','speaker', 'SC','SF', 'SB', 'ZCR'])

# Loop over each audio file in the data directory
for audio_file in os.listdir(data_dir):
    # Try to load the file and parse all of its contents.
    # If something goes wrong, Python will execute the contents of the "except"
    # codeblock.
    try:
        # Use the same string parsers from before and grab info from filename
        digit = int(re.match(re_digit, audio_file)[0][:-1])
        speaker = re.search(re_speaker, audio_file)[0][1:-1]

        # Read in the audio file
        full_path_to_audio_file = os.path.join(data_dir, audio_file)
        (sample_rate, y) = scipy.io.wavfile.read(full_path_to_audio_file)
        
        # If the audio recording is in stereo (2-channels), just use one of them
        if len(y.shape) == 2:
            y = y[:,0]

        # Grab the number of samples, compute the sample interval, and generate
        # the time-stamps for each of the audio samples
        num_samples = len(y)
        sample_interval = 1.0/sample_rate
        t = np.arange(0, num_samples/sample_rate, sample_interval)

        # Compute the Fourier Transform of the audio signal
        ft = np.fft.fft(y)

        # Calculate the audio features to be stored in the data frame
        sc = spectral_centroid(ft, sample_rate)
        sf = spectral_flatness(ft)
        sb = feature1_extractor(y, sample_rate)
        zcr = feature2_extractor(y, sample_rate)
        ### OPTIONAL: ADD MORE FEATURES HERE!
        ###
        ### NOTE: IF YOUR FEATURE EXTRACTOR RETURNS MORE THAN ONE VALUE, THEN
        ### YOU WILL NEED TO SEPARATE THOSE OUT INTO INDIVIDUAL SCALAR VARIABLES

        # Add the info for this file to our dataframe
        feature_dict = {'digit':digit, 'speaker':speaker,'SC':sc,'SF':sf,
                        'SB':sb, # <-- UPDATE FEATURE NAME
                        'ZCR':zcr} # <-- UPDATE FEATURE NAME
                                             # OPTIONAL: EXTEND THE DICTIONARY
                                             # WITH EVEN MORE FEATURES!
        df = df.append(feature_dict, ignore_index=True)

    except Exception as err:
        # Something went wrong!  =(
        # Notife the user of the audio file that broke and what the error was
        print(audio_file)
        raise err

In [ ]:
# Print the head of the dataframe
print('df.head():\n')
print(df.head(15))

# Also print some summary information about the dataframe
print('\n\ndf.info():\n')
print(df.info())

df.head():

   digit   speaker           SC        SF  \
0      8   nicolas  1160.920727  0.480644   
1      5     lucas   951.610951  0.293918   
2      1   nicolas  1179.713806  0.528961   
3      2   jackson   794.259527  0.253679   
4      6    george  1939.407395  0.348809   
5      2   nicolas  1011.517350  0.398295   
6      0   jackson   802.618018  0.287560   
7      1   jackson  1096.281780  0.366212   
8      5   nicolas  1082.734343  0.502244   
9      7    george  1424.554517  0.406677   
10     7      theo  1275.965936  0.460095   
11     4      theo   732.173447  0.256113   
12     9  yweweler   969.760340  0.341572   
13     7  yweweler  1290.708710  0.556307   
14     8    george  1949.068986  0.445941   

                                                   SB     ZCR  
0   [1005.2042014503768, 997.845812322174, 990.288...   453.0  
1   [673.1580690993023, 649.2553059135399, 651.293...  1094.0  
2   [1137.0018355535851, 1070.6516533036106, 1105....   456.0  
3   [1215.3

In [ ]:
df['SB']

0       [1005.2042014503768, 997.845812322174, 990.288...
1       [673.1580690993023, 649.2553059135399, 651.293...
2       [1137.0018355535851, 1070.6516533036106, 1105....
3       [1215.3599435293595, 1154.4935138098194, 998.5...
4       [1176.495822602282, 1166.7805223444323, 1158.6...
                              ...                        
2995    [1116.8594926740732, 1122.1291531382694, 1131....
2996    [1376.4465611714995, 1303.2980519903265, 1155....
2997    [986.4807288827894, 928.9289689294865, 818.828...
2998    [778.1733253558455, 737.8005312386919, 751.030...
2999    [968.2708698600595, 958.798358596892, 951.6886...
Name: SB, Length: 3000, dtype: object

## Considering Your Features

Now that we have successfully extracted a bunch of features from the audio files in this database, let's pause and examine the results.

In [ ]:
# Visualize a pairplot of the spectral features. In this instance, color each
# datapoint by the *person* speaking the digit to see if each *speaker* has a
# distinguishable set of spectral characteristics (e.g., deep vs high voice)
g = sns.pairplot(data=df, vars=['SC', 'SF', 'SB', 'ZCR'],
                 hue='speaker', corner=True)
plt.suptitle('Spectral Features (colored by speaker)', x=0.5, y=1.02)

# Clean up the plot appearance
plt.gcf().tight_layout()

# Repeat this plot, but now color each datapoint by the *digit* being spoken to
# see if each *word* has a distinguishable set of spectral characteristics
# (e.g., does the buzzing 'z' in 'zero' cause a different set of characteristics
# than the shap 't' sound in 'two'?)
g = sns.pairplot(data=df, vars=['SC', 'SF', 'SB', 'ZCR'],
                 hue='digit', corner=True)
plt.suptitle('Spectral Features (colored by digit)', x=0.5, y=1.02)

# Clean up the plot appearance
plt.gcf().tight_layout()

## Problem 3

Does it look like you have engineered some features that help separate the audio files by which digit was being spoken? What about by which person was speaking?

SC, SF, and SB are the best for separating audio files by digit and person.

You can re-jigger your feature extractors however you want (or add more), and re-run the above cells to examine those effects on the feature space.

# Train Machine Learning Algorithm

The following section perfectly parallels the content from the tutorial. We are simply copying it over here to the exercises so that you can see if your features helped classify the audio data.

You are not *quite* done yet though. There are some thoughts in the "Conclusion" section you may want to consider.

In [ ]:
# Select the name of the column to be used as the desired 'label' output
# (For these files, it makes sense to use either 'speaker' or 'digit')
label = 'speaker'

# Create a list of features to be used in the classification process.
# We use the 'set' object in Python to quickly remove the unwanted columns
#
# NOTE: Add any columns you *do not* want to be included in the training
# algorithm to the "columns_to_remove" variable.
columns_to_remove = set([label, 'file', 'trial'])
features = set(df.columns) - columns_to_remove
features = list(features)

print('Classifying {} using...'.format(label))
print('features: {}'.format(features))

In [ ]:
# Convert the data in the 'speaker', 'digit', and 'trial' columns of the
# dataframe into the 'Categorical' type.
df.speaker = pd.Categorical(df.speaker)
df.digit = pd.Categorical(df.digit)

In [ ]:
# If the speaker is in the set of features used to classify the audio files,
# then the string speaker value should be converted to its numerical version.
if 'speaker' in features:
    # Add a column containing the numerically encoded version of speaker name
    df['speaker_code'] = df.speaker.cat.codes

    # Replace the 'speaker' feature with the 'speaker_code' feature
    features.remove('speaker')
    features.append('speaker_code')

    # Double check that there are not duplicate entries in the list of features
    # by converting to a `set` object (which automatically removes duplicate
    # entries) then convert back `list` object.
    features = list(set(features))

# Update the user on what columns and content remain in the datafram
df.info()

Next we split the data into training and test sets.

In [ ]:
# split X and y into training and testing sets
# NOTE: Using convention that X = features, y = label
X_train, X_test, y_train, y_test = train_test_split(
    df.loc[:,features], df[label], test_size=0.25,
    random_state=0, stratify = df[label])

Now, with the correct training features and labels, we can train a machine learning classification model. To start with, let's consider a decision tree.

In [ ]:
# Instantiate a DecisionTree model
model = tree.DecisionTreeClassifier(max_depth=4)

# Fit the model to the training data
model.fit(X_train, y_train)

In [ ]:
# Convert each class name into a string representation
class_names = [str(c) for c in model.classes_]

# Generate the data to visualize the decision tree
dot_data = tree.export_graphviz(model, out_file=None, 
                         feature_names=features,  
                         class_names=class_names,  
                         filled=True, rounded=True) 

# Visualize the tree from that exported data
graph = graphviz.Source(dot_data)  
graph

## Evaluate on Test Data

In [ ]:
# Visualize the confusion matrix
def plot_cmatrix(cm,labels,title='Confusion Matrix'):
    """
    Plot the confusion matrix for the classifier

    :param cm: the actual confusion matrix
    :param labels: the labels to add along the axes of the matrix
    :param title: the title to place over the confusion matrix plot
    """
    # Generate a new figure and axes instance
    fig = plt.figure()
    ax = fig.add_subplot(111)

    # Display the confusion matrix (attempt to account for normalization)
    row_totals = np.sum(cm, axis=1)
    vmax_val = row_totals.max() if row_totals.max() > 1 else 1.0
    ax_im = ax.imshow(cm, cmap='Reds', vmin=0, vmax=vmax_val)

    # Annotate the figure
    plt.title(title, fontsize=20)
    fig.colorbar(ax_im)
    ax.set_xticks(range(len(labels)))
    ax.set_yticks(range(len(labels)))
    ax.set_xticklabels(labels, fontsize=16, rotation=70)
    ax.set_yticklabels(labels, fontsize=16)
    plt.xlabel('Predicted', fontsize=16)
    plt.ylabel('True', fontsize=16)
    plt.show()

In [ ]:
# Predict class label probabilities
labels=np.sort(y_test.unique())
y_test_pred = model.predict(X_test)

cm = metrics.confusion_matrix(y_test,y_test_pred, labels=labels)
plot_cmatrix(cm, labels)
cm

# Conclusion

For consistency with the tutorial, we have used a decision tree to classify the data, but you could train a *different* classifier. In particular, do you recall if there is a classifier that may help separate features that are all mingled together like some of these features are?

SVM

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7a947772-3ac4-4afb-803e-f093dcf9043f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>